In [13]:
#!/usr/bin/env python

#SBATCH --job-name=PowerEva

#SBATCH --error=%x.%j.err
#SBATCH --output=%x.%j.out

#SBATCH --mail-user=hzhao@teco.edu

#SBATCH --export=ALL

#SBATCH --time=48:00:00

#SBATCH --partition=sdil
#SBATCH --gres=gpu:1

import sys
import os
from pathlib import Path
import pickle
import torch
import pprint
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
from utils import *
from configuration import *
import numpy as np

if not os.path.exists('./evaluation/'):
    os.makedirs('./evaluation/')

In [14]:
args = parser.parse_args([])
args.DATASET = 0
args.SoftEva = True
args = FormulateArgs(args)

In [15]:
valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')
pprint.pprint(datainfo)

{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_time': 32,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}


In [16]:
float('nan')

nan

In [17]:
MODELS = ['MLP', 'pNN', 'pNN_opw', 'LSTM', 'SNN', 'SNN_opw']
models_t = ['LSTM', 'SNN', 'SNN_opw']
models_n = ['MLP', 'pNN', 'pNN_opw']
learnablability = [[False, False], [True, False], [True, True]]
results = torch.zeros([10,6,3,4])

for seed in range(10):
    for m, mo in enumerate(MODELS):
        for l, lnc in enumerate(learnablability):
            args.SEED = seed

            if mo in models_t:
                args.task = 'temporized'
            elif mo in models_n:
                args.task = 'normal'
                
            if mo.startswith('SNN'):
                args.metric = 'acc_snn'
            else:
                args.metric = 'acc'

            valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
            test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')

            modelname = f"baseline_model_{mo}_data_{datainfo['dataname']}_seed_{args.SEED:02d}_lnc_{lnc[0]}_ilnc_{lnc[1]}.model"
            # print(modelname)
            
            if os.path.isfile(f'./models/{modelname}'):                
                model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
                model.UpdateArgs(args)

                SetSeed(args.SEED)

                evaluator = Evaluator(args).to(args.DEVICE)

                for x,y in valid_loader:
                    X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
                for x,y in test_loader:
                    X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

                acc_valid, power_valid = evaluator(model, X_valid, y_valid)
                acc_test, power_test   = evaluator(model, X_test,  y_test)

                results[seed,m,l,0] = acc_valid
                results[seed,m,l,1] = power_valid.cpu().item()
                results[seed,m,l,2] = acc_test
                results[seed,m,l,3] = power_test.cpu().item()
            else:
                results[seed,m,l,:] = float('nan')
                
# torch.save(results, f"./evaluation/result_data:{datainfo['dataname']}")


In [18]:
results.shape

torch.Size([10, 6, 3, 4])

In [19]:
m = torch.tensor(np.nanmean(results.numpy(), axis=0))
s = torch.tensor(np.nanstd(results.numpy(), axis=0))

/scratch/slurm_tmpdir/job_22351573/ipykernel_793480/1855993405.py:1: RuntimeWarning: Mean of empty slice
  m = torch.tensor(np.nanmean(results.numpy(), axis=0))


In [ ]:
m.shape

In [9]:
m[4,:,2]

tensor([0.9091,    nan, 0.8377])

In [10]:
s[4,:,2]

tensor([0., nan, 0.])

In [11]:
m[5,:,2]

tensor([nan, nan, nan])

In [12]:
s[5,:,2]

tensor([nan, nan, nan])